In [76]:
import numpy as np
import pandas as pd

def preprocess_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df[(df != "******").all(axis=1)]
    
    col_map = {
        "długośćmm": "dlugoscmm",
        "szerokośćmm": "szerokoscmm",
        "kąt": "kat",
        "obwódmm": "obwodmm",
        "średnica_fereta_hmm": "srednica_fereta_hmm",
        "średn._martinamm": "sredn._martinamm",
        "średn._sitowamm": "sredn._sitowamm",
        "średnia_jaskrawość": "srednia_jaskrawosc"
    }

    if "długośćmm" in df.columns:
        df = df.rename(columns=col_map)
        if "szerokoscmm" in df.columns:
            df["szerokoscmm"] = df["szerokoscmm"].astype(np.float64)

    return df


In [77]:
import pandas as pd
import cv2
import numpy as np

def process_one_image(CSV_PATH = 'KolejnyTest/5M/x/report_5_x.csv',IMAGE_PATH = f'KolejnyTest/5M/x/a.jpg'   ,OUTPUT_PATH = f'KolejnyTest/5M/x/a_marked.jpg'):
    # === CONFIG ===
            # Path to the original image
        # Where to save the output image

    # === LOAD IMAGE AND DATA ===
    img = cv2.imread(IMAGE_PATH)
    df = preprocess_df(pd.read_csv(CSV_PATH))

    height, width = img.shape[:2]

    # === DRAW OBJECTS ===
    for i, row in df.iterrows():
        try:
            # Extract pixel coordinates
            x = int(row['xpix.'])
            y = int(row['ypix.'])
            
            # Extract size in mm → convert to pixels if needed (assuming 1mm = 10px for now, adjust as needed)
            width_mm = float(row['szerokoscmm'])
            height_mm = float(row['dlugoscmm'])
            # angle = 90-float(row['kat'])  # OpenCV uses negative angle for rotation
            angle = float(row['kat'])  # OpenCV uses negative angle for rotation

            mm_to_px = 10  # ⬅️ Adjust this scaling factor based on your image resolution

            width_px = int(width_mm * mm_to_px)
            height_px = int(height_mm * mm_to_px)

            # Draw ellipse representing the object
            center = (x, height-y)
            # axes = (1, 1)
            axes = (width_px // 2, height_px // 2)
            color = (0, 255, 0)  # Green
            thickness = 1

            cv2.ellipse(img, center, axes, angle, 0, 360, color, thickness)

            # Optional: draw ID
            cv2.putText(img, str(int(row['nr.'])), (x + 5, height - y - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
        except Exception as e:
            print(e)

    # === SAVE RESULT ===
    cv2.imwrite(OUTPUT_PATH, img)
    print(f"Image saved to {OUTPUT_PATH}")

In [78]:
# for i in range(1,11):
#     process_one_image(CSV_PATH=f'raport_generating_test/x/results/a{i}_raport.csv',IMAGE_PATH=f'raport_generating_test/x/a{i}.jpg',OUTPUT_PATH=f'raport_generating_test/x/results/a{i}_marked_elipse.jpg')

In [79]:
for i in range(1,11):
    process_one_image(CSV_PATH=f'raport_generating_test/y/results/a{i}_raport_no_model.csv',IMAGE_PATH=f'raport_generating_test/y/a{i}.jpg',OUTPUT_PATH=f'raport_generating_test/y/results/a{i}_marked_elipse.jpg')

Image saved to raport_generating_test/y/results/a1_marked_elipse.jpg
Image saved to raport_generating_test/y/results/a2_marked_elipse.jpg
Image saved to raport_generating_test/y/results/a3_marked_elipse.jpg
Image saved to raport_generating_test/y/results/a4_marked_elipse.jpg
Image saved to raport_generating_test/y/results/a5_marked_elipse.jpg
Image saved to raport_generating_test/y/results/a6_marked_elipse.jpg
Image saved to raport_generating_test/y/results/a7_marked_elipse.jpg
Image saved to raport_generating_test/y/results/a8_marked_elipse.jpg
Image saved to raport_generating_test/y/results/a9_marked_elipse.jpg
Image saved to raport_generating_test/y/results/a10_marked_elipse.jpg


In [80]:
# for i in range(1,11):
#     process_one_image(CSV_PATH=f'raport_generating_test/y/agata_1_y_real_correct_index/raport_a{i}.csv',IMAGE_PATH=f'raport_generating_test/y/a{i}.jpg',OUTPUT_PATH=f'raport_generating_test/y/agata_1_y_real_correct_index/a{i}_marked_elipse_predicted.jpg')